<a href="https://colab.research.google.com/github/tttequila/Kaggle_20Q/blob/main/LLM_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

%pwd

Mounted at /content/drive


'/content'

In [2]:
%%bash
pip install -q -U torch immutabledict sentencepiece
# pip install -q -U keras-nlp
# pip install -q -U keras>=3


mkdir ~/.kaggle
# change the first path to your path of kaggle.json
cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
chmod 600 ~/.kaggle/kaggle.json

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 741.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 53.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.1/168.1 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 74.6 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.3.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.
torchvision 0.18.0+cu121 requires torch==2.3.0, but you have torch 2.3.1 which is incompatible.


### Loading mode

In [3]:
# %%bash
!git clone https://github.com/google/gemma_pytorch.git

# !mkdir /kaggle/working/gemma/
# !mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/gemma/

Cloning into 'gemma_pytorch'...
remote: Enumerating objects: 218, done.
remote: Counting objects: 100% (150/150), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 218 (delta 87), reused 86 (delta 55), pack-reused 68
Receiving objects: 100% (218/218), 2.18 MiB | 23.06 MiB/s, done.
Resolving deltas: 100% (114/114), done.


In [4]:
# login the kaggle (need to store you kaggle.json to your google dirve)
import kagglehub
kagglehub.login()

In [5]:
# Choose variant and machine type
VARIANT = '7b-it-quant'
MACHINE_TYPE = 'cuda'

import os

# Load model weights
weights_dir = kagglehub.model_download(f'google/gemma/pyTorch/{VARIANT}')

# Ensure that the tokenizer is present
tokenizer_path = os.path.join(weights_dir, 'tokenizer.model')
assert os.path.isfile(tokenizer_path), 'Tokenizer not found!'
# Ensure that the checkpoint is present
ckpt_path = os.path.join(weights_dir, f'gemma-{VARIANT}.ckpt')
assert os.path.isfile(ckpt_path), 'PyTorch checkpoint not found!'

Attaching model 'google/gemma/pyTorch/7b-it-quant' to your Colab notebook...


In [11]:
import sys
sys.path.append("gemma_pytorch")
from gemma_pytorch.gemma.config import get_config_for_7b, get_config_for_2b, Architecture
from gemma_pytorch.gemma.model import GemmaForCausalLM
import contextlib, torch

@contextlib.contextmanager
def _set_default_tensor_type(dtype: torch.dtype):
  """Sets the default torch dtype to the given dtype."""
  torch.set_default_dtype(dtype)
  yield
  torch.set_default_dtype(torch.float)

In [ ]:
print(torch.__version__)

In [26]:
import torch
import gemma_pytorch

# Set up model config.
model_config = get_config_for_7b()
model_config.tokenizer = tokenizer_path
model_config.quant = 'quant' in VARIANT
model_config.architecture = gemma_pytorch.gemma.config.Architecture.GEMMA_2

print(model_config.architecture)

for i in range(model_config.num_hidden_layers):
  print(model_config.architecture == gemma_pytorch.gemma.config.Architecture.GEMMA_1)

# Instantiate the model and load the weights.
# torch.set_default_dtype(model_config.get_dtype())
device = torch.device(MACHINE_TYPE)
with _set_default_tensor_type(model_config.get_dtype()):
  model = GemmaForCausalLM(model_config)
  model.load_weights(ckpt_path)
  model = model.to(device).eval()

Architecture.GEMMA_2
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False


ValueError: Unknown architecture: Architecture.GEMMA_2